# AUC画图工具箱

In [2]:
import pickle
import numpy as np
import gzip
import json
from sklearn.metrics import roc_curve, auc

In [3]:
import matplotlib as mpl

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.use14corefonts'] = False
# mpl.rcParams['pdf.usecorefonts'] = True
mpl.rcParams['pdf.compression'] = 9

import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'nature'])

In [4]:
def plot_average_roc_curve(
    predict_result_list:list,
    path_to_: str = None,
    multi_dim: int = None
):
    plt.figure(figsize=(19.2 / 4, 10.8 / 4))
    plt.axis('square')

    tprs = []
    base_fpr = np.linspace(0, 1, 101)

    for item, color in zip(predict_result_list, ["#3e312230", "#f1d14e30", "#4fa65930", "#bc465930", "#854eb930"]):
        pred = item[0]
        target = item[1]
        if multi_dim is not None:
            pred = item[0][:, multi_dim]

        fpr, tpr, thresholds = roc_curve(target, pred)
        roc_auc = auc(fpr, tpr)

        tprs.append(np.interp(base_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        # plt.plot(
        #     fpr, tpr, color=color, lw=1,
        #     # label='ROC curve (area = %0.2f)' % roc_auc
        # )
        tpr = np.interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tprs.append(tpr)

    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std

    plt.plot(
        base_fpr, mean_tprs, color='red', lw=2,
        label='ROC curve (area = %0.2f)' % auc(base_fpr, mean_tprs)
    )

    # plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)

    # plt.plot([0, 1], [0, 1], color='#3f8bc4', lw=1, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) curve')
    plt.legend(loc="lower right")

    if path_to_ is not None:
        plt.savefig(f"{path_to_}", transparent=True)
    plt.clf()

    return {"fpr": base_fpr, "tpr": mean_tprs}

In [5]:
def plot_normal_roc_curve(
    target: np.ndarray,
    pred: np.ndarray,
    path_to_: str = None,
    multi_dim: int = None
):
    if multi_dim is not None:
        pred = pred[:, multi_dim]

    fpr, tpr, thresholds = roc_curve(target, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(19.2 / 4, 10.8 / 4))
    plt.axis('square')
    plt.plot(
        fpr, tpr, color='red', lw=2,
        label='ROC curve (area = %0.2f)' % roc_auc
    )
    # plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) curve')
    plt.legend(loc="lower right")

    if path_to_ is not None:
        plt.savefig(f"{path_to_}", transparent=True)
    plt.clf()

    return {"fpr": fpr, "tpr": tpr}

In [13]:
def plot_multi_roc_curve(
    fpr_tpr_pair: list, # fpr, tpr, color
    path_to_: str,
):

    plt.figure(figsize=(19.2 / 4, 10.8 / 4))
    plt.axis('square')

    for pair in fpr_tpr_pair:
        pair = {"color": "red", "label": "", "linestyle": '-'} | pair
        fpr = pair['fpr']
        tpr = pair['tpr']
        roc_auc = auc(fpr, tpr)
        plt.plot(
            fpr, tpr, color=pair['color'], lw=2, linestyle=pair['linestyle'],
            label=pair['label'] + ' (AUC = %0.2f)' % roc_auc
        )

    # plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) curve')
    plt.legend(loc="lower right")

    plt.savefig(f"{path_to_}", transparent=True)
    plt.clf()

    return None

# 单图

### T1 Voting

In [8]:
S_score_pkl_path = "out/libfeatureselection/T1/model/Onehot/1125899906842624/LogisticRegression_score.pkl"
score_pkl_path = "out/libfeatureselection/voting/T1/T1-TT-AAC_Voting_score.pkl"
stacker_score_pkl_path = "out/libfeatureselection/T1/T1SEstacker/T1-TT-rStacker_score.pkl"

plot_output_pdf_CV_path = "out/libfeatureselection/voting/T1/T1-TT-AAC_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T1/T1-TT-AAC_Voting_CT.pdf"

stacker_score = pickle.load(gzip.open(stacker_score_pkl_path, "rb"))
S_loaded_score = pickle.load(gzip.open(S_score_pkl_path, "rb"))
loaded_score = pickle.load(gzip.open(score_pkl_path, "rb"))

voting_data_CV = plot_average_roc_curve(
    predict_result_list=loaded_score['best_5C_predicted_pair'],
    path_to_=plot_output_pdf_CV_path
)
voting_data_TT = plot_normal_roc_curve(
    target=loaded_score['best_predicted_pair'][1],
    pred=loaded_score['best_predicted_pair'][0],
    path_to_=None
)
stacker_data_TT = plot_normal_roc_curve(
    target=stacker_score['best_predicted_pair'][1],
    pred=stacker_score['best_predicted_pair'][0][:, 1],
    path_to_=None
)
S_data_TT = plot_normal_roc_curve(
    target=S_loaded_score['best_predicted_pair'][1],
    pred=S_loaded_score['best_predicted_pair'][0][:, 1],
    path_to_=None
)

plot_multi_roc_curve(
    fpr_tpr_pair=[
        voting_data_CV | {"color": "#dd1940", "label": "AAC-Voting, CV"},
        voting_data_TT | {"color": "#1f4aec", "label": "AAC-Voting, TT"},
        S_data_TT | {"color": "#2c5618", "label": "AAC-LR, TT"},
        stacker_data_TT | {"color": "#7514f5", "label": "T1SEstacker, TT"},
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

### T1-Voting & T1SEstacker in RTX

In [9]:
score_pkl_path = "out/libfeatureselection/voting/T1/T1-RTX-AAC_Voting_score.pkl"
stacker_score_pkl_path = "out/libfeatureselection/T1/T1SEstacker-RTX/T1-RTX-rStacker_score.pkl"

plot_output_pdf_CV_path = "out/libfeatureselection/voting/T1/T1-RTX-AAC_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T1/T1-RTX-AAC_Voting_CT.pdf"

stacker_score = pickle.load(gzip.open(stacker_score_pkl_path, "rb"))
loaded_score = pickle.load(gzip.open(score_pkl_path, "rb"))

voting_data_CV = plot_average_roc_curve(
    predict_result_list=loaded_score['best_5C_predicted_pair'],
    path_to_=plot_output_pdf_CV_path
)
voting_data_TT = plot_normal_roc_curve(
    target=loaded_score['best_predicted_pair'][1],
    pred=loaded_score['best_predicted_pair'][0],
    path_to_=None
)
stacker_data_TT = plot_normal_roc_curve(
    target=stacker_score['best_predicted_pair'][1],
    pred=stacker_score['best_predicted_pair'][0][:, 1],
    path_to_=None
)

plot_multi_roc_curve(
    fpr_tpr_pair=[
        voting_data_CV | {"color": "#dd1940", "label": "AAC-Voting, RTX: non-RTX"},
        stacker_data_TT | {"color": "#1f4aec", "label": "T1SEstacker, RTX: non-RTX"},
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

### T2 Voting QSO

In [14]:
S_score_pkl_path = "out/libfeatureselection/T2/model/Onehot/2/GaussianProcessClassifier_score.pkl"
score_pkl_path = "out/libfeatureselection/voting/T2/T2-TT-QSO_Voting_score.pkl"
plot_output_pdf_CV_path = "out/libfeatureselection/voting/T2/T2-TT-QSO_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T2/T2-TT-QSO_Voting_CT.pdf"
S_loaded_score = pickle.load(gzip.open(S_score_pkl_path, "rb"))
loaded_score = pickle.load(gzip.open(score_pkl_path, "rb"))

voting_data_CV = plot_average_roc_curve(
    predict_result_list=loaded_score['best_5C_predicted_pair'],
    path_to_=plot_output_pdf_CV_path
)
voting_data_TT = plot_normal_roc_curve(
    target=loaded_score['best_predicted_pair'][1],
    pred=loaded_score['best_predicted_pair'][0],
    path_to_=None
)
S_data_TT = plot_normal_roc_curve(
    target=S_loaded_score['best_predicted_pair'][1],
    pred=S_loaded_score['best_predicted_pair'][0][:, 1],
    path_to_=None
)
plot_multi_roc_curve(
    fpr_tpr_pair=[
        voting_data_CV | {"color": "#dd1940", "label": "QSO-Voting, CV"},
        voting_data_TT | {"color": "#1f4aec", "label": "QSO-Voting, TT"},
        S_data_TT | {"color": "#71c6b1", "label": "QSO-GP, TT"},
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

### T2 Voting 13G

In [15]:
S_score_pkl_path = "out/libfeatureselection/T2/model/Bayes/155100983136298/SVC_score.pkl"
score_pkl_path = "out/libfeatureselection/voting/T2/T2-TT-13G_Voting_score.pkl"

plot_output_pdf_CV_path = "out/libfeatureselection/voting/T2/T2-TT-13G_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T2/T2-TT-13G_Voting_CT.pdf"

xgb_tt_json_path = "/home/georgezhao/Source/From_XiaoXin/TxSEpp_report/out/T2/model/T4SEXGB/tt_model.json"
t4ef_tt_json_path = "/home/georgezhao/Source/From_XiaoXin/TxSEpp_report/out/T2/model/T4EffPred/tt_model.json"
bas6_tt_json_path = "/home/georgezhao/Source/From_XiaoXin/TxSEpp_report/out/T2/model/Bastion6/tt_model.json"

S_loaded_score = pickle.load(gzip.open(S_score_pkl_path, "rb"))
loaded_score = pickle.load(gzip.open(score_pkl_path, "rb"))
xgb_score = json.load(open(xgb_tt_json_path, "r", encoding="UTF-8"))
t4ef_score = json.load(open(t4ef_tt_json_path, "r", encoding="UTF-8"))
bas6_score = json.load(open(bas6_tt_json_path, "r", encoding="UTF-8"))

voting_data_CV = plot_average_roc_curve(
    predict_result_list=loaded_score['best_5C_predicted_pair'],
    path_to_=plot_output_pdf_CV_path
)
voting_data_TT = plot_normal_roc_curve(
    target=loaded_score['best_predicted_pair'][1],
    pred=loaded_score['best_predicted_pair'][0],
    path_to_=None
)
S_data_TT = plot_normal_roc_curve(
    target=S_loaded_score['best_predicted_pair'][1],
    pred=S_loaded_score['best_predicted_pair'][0][:, 1],
    path_to_=None
)
xgb_data_TT = plot_normal_roc_curve(
    target=xgb_score[0]['testing']['origin']['label'],
    pred=xgb_score[0]['testing']['origin']['pred'],
    path_to_=None
)
bas6_data_TT = plot_normal_roc_curve(
    target=bas6_score[0]['testing']['origin']['label'],
    pred=bas6_score[0]['testing']['origin']['pred'],
    path_to_=None
)
t4ef_data_TT = plot_normal_roc_curve(
    target=t4ef_score[0]['testing']['origin']['label'],
    pred=t4ef_score[0]['testing']['origin']['pred'],
    path_to_=None
)
plot_multi_roc_curve(
    fpr_tpr_pair=[
        voting_data_CV | {"color": "#dd1940", "label": "13Groups-Voting, CV"},
        voting_data_TT | {"color": "#1f4aec", "label": "13Groups-Voting, TT"},
        S_data_TT | {"color": "#ed702d", "label": "13Groups-SVM, TT"},
        xgb_data_TT | {"color": "#2c5618", "label": "T4SEXGB, TT"},
        t4ef_data_TT | {"color": "#e9f44f", "label": "T4EffPred, TT"},
        bas6_data_TT | {"color": "#7514f5", "label": "Bastion6, TT"},
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

### T2-13G-Voting & Other in SP

In [6]:
plot_output_pdf_CV_path = "out/libfeatureselection/voting/T2/T2-SP-13G_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T2/T2-SP-13G_Voting_CT.pdf"

score_pkl_path = "out/libfeatureselection/voting/T2/T2-SP-13G_Voting_score.pkl"
xgb_tt_path = "out/libfeatureselection/T2/T4SEXGB_SP/T4SEXGB_score.pkl"
t4ef_tt_path = "out/libfeatureselection/T2/T4EffPred_SP/T4EffPred_score.pkl"
bas6_tt_path = "out/libfeatureselection/T2/Bastion6_SP/Bastion6_score.pkl"

loaded_score = pickle.load(gzip.open(score_pkl_path, "rb"))
xgbloaded_score = pickle.load(gzip.open(xgb_tt_path, "rb"))
t4efloaded_score = pickle.load(gzip.open(t4ef_tt_path, "rb"))
bas6loaded_score = pickle.load(gzip.open(bas6_tt_path, "rb"))

voting_data_TT = plot_normal_roc_curve(
    target=loaded_score['best_predicted_pair'][1],
    pred=loaded_score['best_predicted_pair'][0],
    path_to_=None
)
xgb_data_TT = plot_normal_roc_curve(
    target=xgbloaded_score['best_predicted_pair'][1],
    pred=xgbloaded_score['best_predicted_pair'][0],
    path_to_=None
)
t4ef_data_TT = plot_normal_roc_curve(
    target=t4efloaded_score['best_predicted_pair'][1],
    pred=t4efloaded_score['best_predicted_pair'][0],
    path_to_=None
)
bas6_data_TT = plot_normal_roc_curve(
    target=bas6loaded_score['best_predicted_pair'][1],
    pred=bas6loaded_score['best_predicted_pair'][0],
    path_to_=None
)

plot_multi_roc_curve(
    fpr_tpr_pair=[
        voting_data_TT | {"color": "#dd1940", "label": "13Groups-Voting, SP: non-SP"},
        t4ef_data_TT | {"color": "#1f4aec", "label": "T4EffPred, SP: non-SP"},
        xgb_data_TT | {"color": "#2c5618", "label": "T4SEXGB, SP: non-SP"},
        bas6_data_TT | {"color": "#7514f5", "label": "Bastion6, SP: non-SP"},
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

### T6 Voting AAC

In [16]:
S_score_pkl_path = "out/libfeatureselection/T6/model/Onehot/1125899906842624/KNeighborsClassifier_score.pkl"
score_pkl_path = "out/libfeatureselection/voting/T6/T6-TT-AAC_Voting_score.pkl"
plot_output_pdf_CV_path = "out/libfeatureselection/voting/T6/T6-TT-AAC_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T6/T6-TT-AAC_Voting_CT.pdf"
S_loaded_score = pickle.load(gzip.open(S_score_pkl_path, "rb"))
loaded_score = pickle.load(gzip.open(score_pkl_path, "rb"))

voting_data_CV = plot_average_roc_curve(
    predict_result_list=loaded_score['best_5C_predicted_pair'],
    path_to_=plot_output_pdf_CV_path
)
voting_data_TT = plot_normal_roc_curve(
    target=loaded_score['best_predicted_pair'][1],
    pred=loaded_score['best_predicted_pair'][0],
    path_to_=None
)
S_data_TT = plot_normal_roc_curve(
    target=S_loaded_score['best_predicted_pair'][1],
    pred=S_loaded_score['best_predicted_pair'][0][:, 1],
    path_to_=None
)
plot_multi_roc_curve(
    fpr_tpr_pair=[
        voting_data_CV | {"color": "#dd1940", "label": "AAC-Voting, CV"},
        voting_data_TT | {"color": "#1f4aec", "label": "AAC-Voting, TT"},
        S_data_TT | {"color": "#71c6b1", "label": "AAC-KNN, TT"},
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

### T6 Voting 15G

In [17]:
S_score_pkl_path = "out/libfeatureselection/T6/model/Bayes/1698039127349256/KNeighborsClassifier_score.pkl"
score_pkl_path = "out/libfeatureselection/voting/T6/T6-TT-15G_Voting_score.pkl"

plot_output_pdf_CV_path = "out/libfeatureselection/voting/T6/T6-TT-15G_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T6/T6-TT-15G_Voting_CT.pdf"

bas6_tt_json_path = "/home/georgezhao/Source/From_XiaoXin/TxSEpp_report/out/T6/model/Bastion6/tt_model.json"
Pyt6_tt_json_path = "/home/georgezhao/Source/From_XiaoXin/TxSEpp_report/out/T6/model/PyPredT6/tt_model.json"

S_loaded_score = pickle.load(gzip.open(S_score_pkl_path, "rb"))
loaded_score = pickle.load(gzip.open(score_pkl_path, "rb"))

pyt6_score = json.load(open(Pyt6_tt_json_path, "r", encoding="UTF-8"))
bas6_score = json.load(open(bas6_tt_json_path, "r", encoding="UTF-8"))

voting_data_CV = plot_average_roc_curve(
    predict_result_list=loaded_score['best_5C_predicted_pair'],
    path_to_=plot_output_pdf_CV_path
)
voting_data_TT = plot_normal_roc_curve(
    target=loaded_score['best_predicted_pair'][1],
    pred=loaded_score['best_predicted_pair'][0],
    path_to_=None
)
S_data_TT = plot_normal_roc_curve(
    target=S_loaded_score['best_predicted_pair'][1],
    pred=S_loaded_score['best_predicted_pair'][0][:, 1],
    path_to_=None
)
bas6_data_TT = plot_normal_roc_curve(
    target=bas6_score[0]['testing']['origin']['label'],
    pred=bas6_score[0]['testing']['origin']['pred'],
    path_to_=None
)
pyt6_data_TT = plot_normal_roc_curve(
    target=pyt6_score[0]['testing']['origin']['label'],
    pred=pyt6_score[0]['testing']['origin']['pred'],
    path_to_=None
)
plot_multi_roc_curve(
    fpr_tpr_pair=[
        voting_data_CV | { "color": "#dd1940", "label": "15Groups-Voting, CV" },
        voting_data_TT | { "color": "#1f4aec", "label": "15Groups-Voting, TT" },
        S_data_TT | { "color": "#ed702d", "label": "15Groups-KNN, TT" },
        bas6_data_TT | { "color": "#2c5618", "label": "Bastion6, TT" },
        pyt6_data_TT | { "color": "#e9f44f", "label": "PyPredT6, TT" },
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

### T6 Voting 15G in AB

In [17]:
plot_output_pdf_CV_path = "out/libfeatureselection/voting/T6/T6-15G_Voting_CV.pdf"
plot_output_pdf_multi_path = "out/libfeatureselection/voting/T6/T6-15G_Voting_CT.pdf"

ab_score_pkl_path = "out/libfeatureselection/voting/T6/T6-AB-15G_Voting_score.pkl"
ab_bas6_tt_path = "out/libfeatureselection/T6/Bastion6_AB/Bastion6_score.pkl"
ab_pyt6_tt_path = "out/libfeatureselection/T6/PyPredT6_AB/PyPredT6_score.pkl"
ae_score_pkl_path = "out/libfeatureselection/voting/T6/T6-AE-15G_Voting_score.pkl"
ae_bas6_tt_path = "out/libfeatureselection/T6/Bastion6_AE/Bastion6_score.pkl"
ae_pyt6_tt_path = "out/libfeatureselection/T6/PyPredT6_AE/PyPredT6_score.pkl"

ab_score_score = pickle.load(gzip.open(ab_score_pkl_path, "rb"))
ab_bas6_score = pickle.load(gzip.open(ab_bas6_tt_path, "rb"))
ab_pyt6_score = pickle.load(gzip.open(ab_pyt6_tt_path, "rb"))
ae_score_score = pickle.load(gzip.open(ae_score_pkl_path, "rb"))
ae_bas6_score = pickle.load(gzip.open(ae_bas6_tt_path, "rb"))
ae_pyt6_score = pickle.load(gzip.open(ae_pyt6_tt_path, "rb"))

ab_score_score_data = plot_normal_roc_curve(
    target=ab_score_score['best_predicted_pair'][1],
    pred=ab_score_score['best_predicted_pair'][0],
    path_to_=None
)
ab_bas6_score_data = plot_normal_roc_curve(
    target=ab_bas6_score['best_predicted_pair'][1],
    pred=ab_bas6_score['best_predicted_pair'][0],
    path_to_=None
)
ab_pyt6_score_data = plot_normal_roc_curve(
    target=ab_pyt6_score['best_predicted_pair'][1],
    pred=ab_pyt6_score['best_predicted_pair'][0],
    path_to_=None
)
ae_score_score_data = plot_normal_roc_curve(
    target=ae_score_score['best_predicted_pair'][1],
    pred=ae_score_score['best_predicted_pair'][0],
    path_to_=None
)
ae_bas6_score_data = plot_normal_roc_curve(
    target=ae_bas6_score['best_predicted_pair'][1],
    pred=ae_bas6_score['best_predicted_pair'][0],
    path_to_=None
)
ae_pyt6_score_data = plot_normal_roc_curve(
    target=ae_pyt6_score['best_predicted_pair'][1],
    pred=ae_pyt6_score['best_predicted_pair'][0],
    path_to_=None
)

plot_multi_roc_curve(
    fpr_tpr_pair=[
        ab_score_score_data | {"color": "#dd1940", "label": "15Groups-Voting, AB: AE"},
        ab_bas6_score_data | {"color": "#1f4aec", "label": "Bastion6, AB: AE"},
        ab_pyt6_score_data | {"color": "#2c5618", "label": "PyPredT6, AB: AE"},
        ae_score_score_data | {"color": "#dd1940", "label": "15Groups-Voting, AE: AB", "linestyle": '--' },
        ae_bas6_score_data | {"color": "#1f4aec", "label": "Bastion6, AE: AB", "linestyle": '--' },
        ae_pyt6_score_data | {"color": "#2c5618", "label": "PyPredT6, AE: AB", "linestyle": '--' },
    ],
    path_to_=plot_output_pdf_multi_path
)

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>

<Figure size 480x270 with 0 Axes>